# Portfolio Optimization

Portfolio Optimization is a powerful tool for selecting a range of assets of varying risk/reward. While portfolio optimization can be used to address a broad set of asset selection tasks, the classic understanding deals in stocks, typically where you optimize your stock portfolio to maximize expected return given a certain tolerance to risk. However your objective function can be changed so that you meet a minimum return while minimizing risk. Simulating this process for various risk tolerance thresholds is a useful process for visualizing the change in expected returns as your ability to tolerate risk increases. 

In this notebook, I used Gurobi Solver to demonstrate portfolio optimization with three theoretical stocks. I set the expected returns, standard deviation of returns, correlation of the stocks, and the covariance matrix associated with the stocks, as well. Instead of using an actual dollar amount as a starting investment amount, the model will distribute a percentage of the total investment to each stock.  

In [26]:
import math
import numpy as np
from gurobipy import *
m = Model('Portfolio_Optimization')

In [27]:
MeanReturns = [0.14, 0.11, 0.1]
SdReturns = [0.2,0.15,0.08]

In [28]:
Corrs = [[1,0.6,0.4],[0.6,1,0.7],[0.4,0.7,1]]

In [29]:
covs = [[0.04, 0.018, 0.0064],[0.018,0.0225,0.0084],[0.0064,0.0084,0.0064]]


I created three variables to serve as the percentage of the initial investment that should be allocated to each stock. 

In [30]:
m.remove(m.getVars())
Inv_Weights = []
for i in range(len(MeanReturns)):
    Inv_Weights.append(m.addVar(vtype=GRB.CONTINUOUS, name = "Stock" + str(i), lb= 0.0))
    
print(m.getVars())
m.update()

[]


In [31]:


m.setObjective(quicksum(Inv_Weights[i] * covs[i][j] * Inv_Weights[j] for i in range(len(Inv_Weights)) for j in range(len(Inv_Weights))), GRB.MINIMIZE)

m.update()

Below I am adding constraints to the optimization model. The first constraint mandates that the sum of the investment weights is equal to 1, i.e. 100% of your money is invested. The second constraint requires a minimum return of 0.12, ie a return of at least 12% on your investments. 

In [32]:
m.remove(m.getConstrs())

Req_return = 0.12

m.addConstr(quicksum(Inv_Weights), GRB.EQUAL, 1)
m.addConstr(quicksum(Inv_Weights[i] * MeanReturns[i] for i in range(len(MeanReturns))), GRB.GREATER_EQUAL,Req_return)

print(m.getConstrs())
m.update()

[]


In [33]:
m.optimize()

Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model has 6 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e-02, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+00]
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 6 nonzeros
Presolved model has 6 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 2
 AA' NZ     : 6.000e+00
 Factor NZ  : 1.000e+01
 Factor Ops : 3.000e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.39490847e+05 -1.39490847e+05  2.50e+03 6.11e-06  1.00e+06     0s
   1   4.91553548e+03 -5.16907979e+03  1.49e+02 3.64e-07  6.17e+04     0s
   2   3.34728926e-02 -3.07903637e+02  1.50e-01 3.67e-10  1.39e+02     0s
   3   2.42930548e-02 -5.49077879e+01  1.50e-07 3.64e-16  1.37e+01     0s

In [34]:
print(m.ObjVal)

0.014800002415706223
